In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra
import wandb
from omegaconf import OmegaConf
import copy

from model.rbm.rbm import RBM
from model.rbm.rbm_torch import RBMtorch

[18:21:40.680] INFO   CaloQuVAE                                         Loading configuration.


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
config=compose(config_name="config.yaml")
wandb.init(tags = [config.data.dataset_name], project=config.wandb.project, entity=config.wandb.entity, config=OmegaConf.to_container(config, resolve=True), mode='disabled')

In [3]:
torch.manual_seed(0)
manual_RBM = RBM(config)
manual_weights = manual_RBM.weight_dict

torch.manual_seed(0)
torch_RBM = RBMtorch(config)
torch_weights = torch_RBM.weight_dict

#check if two weight dicts are equal
assert manual_weights.keys() == torch_weights.keys()
for key in manual_weights.keys():
    assert torch.all(manual_weights[key] == torch_weights[key])
# print top values in both weight dicts
# print("Manual RBM weights:")
# print(manual_weights['02'][0])
# print("Torch RBM weights:")
# print(torch_weights['02'][0])

# calculate mock gradients
# manual_RBM.calculate_mock_gradient()
# torch_RBM.calculate_mock_gradient()

#generate dummy post_samples
torch.manual_seed(0)

n_samples = 10
n_nodes_p = config.rbm.latent_nodes_per_p
post_samples = [
    torch.randint(0, 2, (n_samples, n_nodes_p), dtype=torch.float32)
    for _ in range(4)
]

# Compute gradients with identical RNG sequences
with torch.random.fork_rng():
    torch.manual_seed(123)
    manual_RBM.gradient_rbm_centered(post_samples)

with torch.random.fork_rng():
    torch.manual_seed(123)
    torch_RBM.gradient_rbm_centered(post_samples)

# Compare with tolerances + diagnostics
for kind in ("bias", "weight"):
    for key in manual_RBM.grad[kind]:
        a = manual_RBM.grad[kind][key]
        b = torch_RBM.grad[kind][key]
        if not torch.allclose(a, b, rtol=1e-5, atol=1e-8):
            diff = (a - b).abs().max().item()
            print(f"{kind}.{key} differ, max|Δ|={diff}")

# use different optimizers
manual_RBM.initOpt()
manual_RBM.update_params()

torch_RBM.initOpt_torch()
torch_RBM.update_params_torch()

# print top values in both weight dicts
manual_weights = manual_RBM._weight_dict
torch_weights = torch_RBM.weight_dict
print("Manual RBM weights:")
print(manual_weights['02'][0])
print("Torch RBM weights:")
print(torch_weights['02'][0])

for i, group in enumerate(torch_RBM.opt.param_groups):
    print(f"Param group {i}")
    for p in group['params']:
        print("  id:", id(p), " shape:", p.shape, " requires_grad:", p.requires_grad)



# check if weights are equal
assert manual_weights.keys() == torch_weights.keys()
for key in manual_weights.keys():
    diff = (manual_weights[key] - torch_weights[key]).abs().max()
    print(f"Max diff for {key}: {diff}")


[18:21:48.393] WARN   model.rbm.zephyr                                  QPU is offline. Setting a hard-coded zephyr. Check to see you're pinging the correct chip_id


/home/leozhu/CaloQuVAE


[18:21:49.361] WARN   model.rbm.zephyr                                  QPU is offline. Setting a hard-coded zephyr. Check to see you're pinging the correct chip_id


/home/leozhu/CaloQuVAE


/usr/local/lib/python3.12/dist-packages/torch/random.py:187: UserWarning: CUDA reports that you have 6 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(message)


Manual RBM weights:
tensor([-2.0857, -0.7343,  0.0906, -0.6137, -2.3928,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.

/usr/local/lib/python3.12/dist-packages/torch/_tensor.py:1128: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/Scalar.cpp:22.)
  return self.item().__format__(format_spec)


In [ ]:
torch.manual_seed(0)
rbm_torch = RBMtorch(config)
# Clone weights before update
before_update = {k: v.clone() for k, v in rbm_torch._weight_dict.items()}

# before step
p = rbm_torch._weight_dict['01']
print(id(p), p.storage().data_ptr())


# Perform optimizer step
rbm_torch.initOpt_torch()
rbm_torch.calculate_mock_gradient()
rbm_torch.update_params_torch()


rbm_torch.update_params_torch()

# Clone weights after update
after_update = {k: v.clone() for k, v in rbm_torch._weight_dict.items()}
# after step: these must be identical
q = rbm_torch._weight_dict['01']
print(id(q), q.storage().data_ptr())

# Check for changes
for k in before_update:
    changed = not torch.allclose(before_update[k], after_update[k])
    print(f"Weights for '{k}' updated: {changed}")


In [ ]:
class testRBM(RBM):
    def calculate_mock_gradient(self):
        """Creates a deterministic mock gradient for testing."""
        self.grad = {"bias": {}, "weight": {}}
        for key, param in self.bias_dict.items():
            self.grad["bias"][key] = torch.ones_like(param) * 0.1
        for key, param in self.weight_dict.items():
            self.grad["weight"][key] = torch.ones_like(param) * -0.05
RBMinitial = testRBM(config)


In [ ]:
rbm_custom = copy.deepcopy(RBMinitial)
rbm_custom.initOpt()
rbm_custom.calculate_mock_gradient()

initial_weight_01 = rbm_custom.weight_dict['01'].data.clone()
rbm_custom.update_params()
final_weight_custom = rbm_custom.weight_dict['01'].data.clone()

print(f"Initial Weight (Top 5 values): {initial_weight_01.flatten()[:5].tolist()}")
print(f"Final Weight (Top 5 values):   {final_weight_custom.flatten()[:5].tolist()}\n")

rbm_torch = copy.deepcopy(RBMinitial)
rbm_torch.initOpt_torch()
rbm_torch.calculate_mock_gradient()

initial_weight_01_torch = rbm_torch.weight_dict['01'].data.clone()
rbm_torch.update_params_torch()
final_weight_torch = rbm_torch.weight_dict['01'].data.clone()

print(f"Initial Weight (Top 5 values): {initial_weight_01_torch.flatten()[:5].tolist()}")
print(f"Final Weight (Top 5 values):   {final_weight_torch.flatten()[:5].tolist()}\n")


In [ ]:
print("--- Running Test 1: Custom AdamOpt (Adam with L2 Regularization) ---")
rbm_custom = copy.deepcopy(rbm_initial)
rbm_custom.initOpt()
rbm_custom.calculate_mock_gradient()

# Store initial weights for comparison
initial_weight_01 = rbm_custom.weight_dict['01'].data.clone()
rbm_custom.update_params()
final_weight_custom = rbm_custom.weight_dict['01'].data.clone()

print(f"Initial Weight (Top 5 values): {initial_weight_01.flatten()[:5].tolist()}")
print(f"Final Weight (Top 5 values):   {final_weight_custom.flatten()[:5].tolist()}\n")


# --- Test 2: PyTorch's Adam (AdamW) Implementation ---
print("--- Running Test 2: PyTorch `torch.optim.Adam` (AdamW) ---")
rbm_torch = copy.deepcopy(rbm_initial)
rbm_torch.initOpt_torch()
rbm_torch.calculate_mock_gradient()

# Store initial weights (should be identical to the other test)
initial_weight_01_torch = rbm_torch.weight_dict['01'].data.clone()
rbm_torch.update_params_torch()
final_weight_torch = rbm_torch.weight_dict['01'].data.clone()

print(f"Initial Weight (Top 5 values): {initial_weight_01_torch.flatten()[:5].tolist()}")
print(f"Final Weight (Top 5 values):   {final_weight_torch.flatten()[:5].tolist()}\n")

# --- Comparison and Conclusion ---
print("--- Comparison ---")
# Check if initial weights are identical
are_initial_weights_same = torch.allclose(initial_weight_01, initial_weight_01_torch)
print(f"Initial weights were identical: {are_initial_weights_same}")

# Check if final weights are identical
are_final_weights_same = torch.allclose(final_weight_custom, final_weight_torch)
print(f"Final weights are identical: {are_final_weights_same}")

if not are_final_weights_same:
    difference = torch.mean(torch.abs(final_weight_custom - final_weight_torch))
    print(f"Average absolute difference between final weights: {difference.item():.8f}")
    print("\nConclusion: The final weights are different, confirming the optimizer behaviors are not identical.")